<a href="https://colab.research.google.com/github/jacobazevedojr/CECS-451-Assignment-3/blob/main/genetic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1075]:
# Genetic Algorithms (Local Search)

# Implement a program that performs the Genetic algorithm with 8
# states including the three operations, i.e., selection, crossover, mutation 
# to find a solution.

from board import Board
import numpy as np
import random
import time

In [1076]:
# Driver

def start(k, n):
  tries = 0
  restarts = 0
  # Initialize k random states of n X n boards
  states = []
  for i in range(k):
    states.append(Board(n))

  noSolution = True
  while noSolution:
    # Initializes fitness values for all state
    # if tries == 1:
    #   print("Initial States")
    #   for i, val in enumerate(states):
    #     val.fitness()
    #     print(val.map, "Fitness: ", val.fit)        
    #     print()

    parents = selection(states)

    # if tries == 1:
    #   print("Selected Parents")
    #   for i, val in enumerate(parents):
    #     val.fitness()
    #     print(val.map, "Fitness: ", val.fit)
    #     print()

    newStates = crossover(parents)

    # if tries == 1:
    #   print("Crossover")
    #   for i, val in enumerate(newStates):
    #     val.fitness()
    #     print(val.map, "Fitness: ", val.fit)
    #     print()

    states = mutation(newStates)

    # if tries == 1:
    #   print("Mutation")
    #   for i, val in enumerate(states):
    #     val.fitness()
    #     print(val.map, "Fitness: ", val.fit)        
    #     print()

    tries += 1

    # Comment out this block to prevent random restarts
    if tries % 100 == 0:
      restarts += 1
      for i in range(k):
        states[i] = Board(n)

    optimalFitness = 0
    for i, val in enumerate(states):
      val.fitness()
      if val.fit == optimalFitness:
        print("Iterations: ", tries)
        print("Restarts: ", restarts)
        return val

In [1077]:
# The more fit a state is, the higher likelihood that it is chosen to mate
# There is a likelihood of high fitness states "breeding" with more than one 
# "partners" and low fitness states not breeding at all

# Pass the current k states and return the parents

def selection(states):

  n = states[0].n_queen
  maxNonAttackingPairs = n * (n-1) // 2
  for state in states:
    # Initialize fitness for each state
    state.fitness()

  # Tracks cumulative weight (for selection)
  cumulativeWeights = []
  # Tracks each state's individual weight
  statesRanges = []

  totalNA = 0

  stateNA = []
  for i, state in enumerate(states):
    # Builds the cumulative weight array 
    # Each array adds the curr percent with the last to determine percent range
    currNonAttacking = maxNonAttackingPairs - states[i].fit
    totalNA += currNonAttacking
    stateNA.append(currNonAttacking)

  thisRange = stateNA[0] / totalNA
  statesRanges.append(thisRange)
  cumulativeWeights.append(thisRange)

  for i, val in enumerate(stateNA[1:]):
    thisRange = val / totalNA
    statesRanges.append(thisRange)
    cumulativeWeights.append(cumulativeWeights[i] + thisRange)

  parents = []
  # Use random number generator to choose who "mates"
  for i in range(len(states) // 2):
    rand1 = random.random()
    rand2 = random.random()
    for j, weight in enumerate(cumulativeWeights):
      # The first time the rand is less than a number, it will fall within that
      # index range
      if rand1 <= weight:
        # The first parent is chosen
        parents.append(states[j])

        # Remove j from the pool for the second parent
        newRanges = statesRanges.copy()
        del newRanges[j]

        # Remove j from the pool in the states array also
        statesIndices = [x for x in range(len(states))]
        del statesIndices[j]

        # Find sum of new ranges
        sum = 0
        for k, val in enumerate(newRanges):
          sum += val
                
        # Normalize all ranges relative to remaining ranges
        for w, val1 in enumerate(newRanges):
          newRanges[w] = val1 / sum

        # Build into cumulative weight list
        for z, val2 in enumerate(newRanges[1:]):
          newRanges[z + 1] = newRanges[z] + val2

        # Check where rand2 lands
        for k, val in enumerate(newRanges):
          if rand2 <= val:
            parents.append(states[statesIndices[k]])
            break
        break
        
  return parents

In [1078]:
# Once the selection of mates has been decided, they must now... do the deed

# Pass the selected parent states (always even) 
# and "breed" them, return their children
def crossover(states):
  children = []

  for i in range(0, len(states), 2):
    parent1 = states[i]
    parent2 = states[i + 1]
    # Every pair of parents (beginning with 0 and 1) mate with each other
    # Each pair of parents make a pair of children (twins)
    tempTwins = mate(parent1, parent2)
    for child in tempTwins:
      children.append(child)
  
  return children

In [1079]:
# A random position within the encoded state representation is "mutated"
# The mutated value will be a random value between 0 and (n_queen - 1)

def mutation(states):
  n = states[0].n_queen
  for state in states:
    randIndex = np.random.randint(0, n - 1)
    state.map[randIndex] = np.random.randint(0, n - 1)
  return states

In [1080]:
# Returns the child for two parents
def mate(parent1, parent2):
  # Determine some arbitrary slicing point
  n = parent1.n_queen
  sliceIndex = np.random.randint(1, n - 1)

  twins = []
  twin1 = Board(parent1.n_queen)
  twin1.map = parent1.map[:sliceIndex] + parent2.map[sliceIndex:]

  twin2 = Board(parent1.n_queen)
  twin2.map = parent2.map[:sliceIndex] + parent1.map[sliceIndex:]

  twins.append(twin1)
  twins.append(twin2)

  return twins

In [1082]:
timeStart = time.time()
solution = start(8,5)
solution.showGrid()
timeEnd = time.time()
print(f"Execution time: {timeEnd - timeStart} seconds")

Iterations:  9
Restarts:  0
[0, 0, 1, 0, 0]
[1, 0, 0, 0, 0]
[0, 0, 0, 1, 0]
[0, 1, 0, 0, 0]
[0, 0, 0, 0, 1]
Execution time: 0.002636432647705078 seconds
